In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# Set parameters for data loading and model training
max_words = 10000  # Only consider the top 10,000 words in the dataset
maxlen = 200  # Limit the length of each review to 200 words

In [3]:
# Load dataset
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_words)

17464789/17464789 [==============================] - 135s 8us/step


In [4]:
# Preprocess the data: Pad sequences to have uniform length
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [5]:
# Define the model architecture
model = Sequential()
model.add(Embedding(max_words, 64, input_length=maxlen))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [6]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=50)

Epoch 1/5
500/500 [==============================] - 413s 813ms/step - loss: 0.4160 - accuracy: 0.8117 - val_loss: 0.3154 - val_accuracy: 0.8699
Epoch 2/5
500/500 [==============================] - 403s 807ms/step - loss: 0.2554 - accuracy: 0.9006 - val_loss: 0.3099 - val_accuracy: 0.8738
Epoch 3/5
500/500 [==============================] - 364s 728ms/step - loss: 0.1925 - accuracy: 0.9269 - val_loss: 0.3477 - val_accuracy: 0.8678
Epoch 4/5
500/500 [==============================] - 414s 828ms/step - loss: 0.1644 - accuracy: 0.9380 - val_loss: 0.3947 - val_accuracy: 0.8558
Epoch 5/5
500/500 [==============================] - 472s 945ms/step - loss: 0.1327 - accuracy: 0.9517 - val_loss: 0.3837 - val_accuracy: 0.8632


In [8]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

782/782 [==============================] - 81s 98ms/step - loss: 0.3837 - accuracy: 0.8632
Test Accuracy: 0.8632400035858154


In [9]:
# Load the word index from the IMDB dataset
word_index = imdb.get_word_index()

# Reverse the word index to map integers to words
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

1/1 [==============================] - 0s 95ms/step
Review 1:
Text: ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? please give this one a miss br br ? ? and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite ? so all you madison fans give this a miss
Sentiment: Negative

Review 2:
Text: psychological ? it's very interesting that robert altman directed this considering the style and structure of his other films still the trademark altman audio style is evident here and there i think what really makes this film work is the brilliant performance by sandy dennis it's definitely one of 

In [10]:
# Function to decode review from integer sequence to text
def decode_review(sequence):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in sequence])

# Choose two reviews from the test set
review1 = X_test[0]
review2 = X_test[1]

In [11]:
# Decode the reviews from integer sequences to text
decoded_review1 = decode_review(review1)
decoded_review2 = decode_review(review2)

In [12]:
# Preprocess the reviews: Pad sequences to have uniform length
review1 = pad_sequences([review1], maxlen=maxlen)
review2 = pad_sequences([review2], maxlen=maxlen)

# Predict sentiment for the reviews
prediction1 = model.predict(review1)[0][0]
prediction2 = model.predict(review2)[0][0]

1/1 [==============================] - 0s 69ms/step


In [13]:
# Define a function to determine sentiment
def determine_sentiment(prediction):
    return "Positive" if prediction >= 0.5 else "Negative"

# Print the results
print("Review 1:")
print("Text:", decoded_review1)
print("Sentiment:", determine_sentiment(prediction1))
print()
print("Review 2:")
print("Text:", decoded_review2)
print("Sentiment:", determine_sentiment(prediction2))

Review 1:
Text: ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? please give this one a miss br br ? ? and the rest of the cast rendered terrible performances the show is flat flat flat br br i don't know how michael madison could have allowed this one on his plate he almost seemed to know this wasn't going to work out and his performance was quite ? so all you madison fans give this a miss
Sentiment: Negative

Review 2:
Text: psychological ? it's very interesting that robert altman directed this considering the style and structure of his other films still the trademark altman audio style is evident here and there i think what really makes this film work is the brilliant performance by sandy dennis it's definitely one of her darker characters but she plays it so perfectly 